In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential, load_model

# 1. Daten einlesen
data = pd.read_csv("/workspaces/codespaces-jupyter/merged_data.csv")  

# 2. Datenvorbereitung
# Entferne Zeilen mit fehlenden Werten
data = data.dropna()

# Features und Ziel definieren
features = ["Temperatur", "Windgeschwindigkeit","Warengruppe", "Wettercode", "KielerWoche"]
target = "Umsatz"

X = data[features]  # Eingabefeatures
y = data[target]    # Zielvariable

# 3. Train/Test-Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #hier muss die richtige Trennung rein

# 4. Skalierung der Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 5. Neuronales Netz definieren
model =Sequential([
    Dense(64, activation='relu', input_dim=X_train_scaled.shape[1]),  # Erste Schicht
    Dense(32, activation='relu'),                                    # Zweite Schicht
    Dense(16, activation='relu'),                                    # Dritte Schicht
    Dense(1, activation='linear')                                    # Ausgabeschicht (Regression)
])

# 6. Modell kompilieren
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])

# 7. Modell trainieren
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=1000,  # Anzahl der Trainingsdurchläufe
    batch_size=32,
    verbose=1
)

# 8. Modell bewerten
loss, mae, mse = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")

# 9. Vorhersagen generieren
predictions = model.predict(X_test_scaled)

# 10. Ergebnisse anzeigen
results = pd.DataFrame({'Actual': y_test.values, 'Predicted': predictions.flatten()})
print(results.head())

# Optional: Ergebnisse speichern
results.to_csv("predicted_results.csv", index=False)